# Visualize pymoo result
Load the pymoo result object saved with pickle and visualize it.

ONOSControllerPlacement needs to be definded to load the result object.
So, execute the following cell first.

In [1]:
#!/usr/bin/env python
import numpy as np
import networkx as nx
import math
import pickle
from pymoo.core.problem import ElementwiseProblem

class ONOSControllerPlacement(ElementwiseProblem):
    def __init__(self, num_nodes, distance_matrix, shortest_paths, graph, **kwargs):
        super().__init__(n_var=2*num_nodes, 
                         n_obj=4, 
                         n_constr=2, 
                         xl=0, xu=1, 
                         **kwargs)
        self.num_nodes = num_nodes
        self.distance_matrix = distance_matrix
        self.shortest_paths = shortest_paths
        self.graph = graph
    
    def _evaluate(self, x, out, *args, **kwargs):
        controller_nodes = x[:self.num_nodes]   # first half is controller placement
        atomix_nodes = x[self.num_nodes:]       # second half is atomix placement


        num_controller = np.sum(controller_nodes)
        num_atomix = np.sum(atomix_nodes)

        # Obj1: Minimize number of contrtoller
        f1 = num_controller

        # Obj2: Minimize number of atomix
        f2 = num_atomix

        # Obj3: Minimize average FSP
        f3 = calculate_FST(self.num_nodes, 
                           controller_nodes, 
                           atomix_nodes, 
                           self.distance_matrix, 
                           self.shortest_paths)
        
        f4 = calculate_BC(self.num_nodes, 
                           controller_nodes, 
                           atomix_nodes, 
                           self.distance_matrix, 
                        #    self.shortest_paths,
                           self.graph)

        # Constr1: The number of controller is equal to or greater than 2
        g1 = 2 - num_controller

        # Constr2: The number of atomix is equal to or greater than 3
        g2 = 3 - num_atomix
        
        # Add the centrality metrix into optimazing objectives:
        # 1. Nearest controller for each switch
        # 2. The number of controlled switches for each controller should be <= limit_num_switches_controlled (limit_num_switches_controlled=int(math.ceil(num_nodes/num_controller)))
        # 3. return value should be the variance for all controller's betweenness centrality
        out["F"] = [f1, f2, f3, f4]
        out["G"] = [g1, g2]


def calculate_FST(num_nodes, controller_nodes, atomix_nodes, distance_matrix, shortest_paths):
    num_controller = np.sum(controller_nodes)
    num_atomix = np.sum(atomix_nodes)
    controller_list = np.nonzero(controller_nodes)[0].tolist()
    atomix_list = np.nonzero(atomix_nodes)[0].tolist()

    if(num_controller == 0 or num_atomix ==0):
        return math.inf

    # find the nearest controller for each switch
    controller_of = []
    for s in range(num_nodes):
        delay = math.inf
        nearest_controller = None
        for c in controller_list:
            if distance_matrix[s][c] < delay:
                delay = distance_matrix[s][c]
                nearest_controller = c
        controller_of.append(nearest_controller)    

    # calculate average delay to atomix nodes from each controller
    average_atomix_delay_from = {}
    for c in controller_list:
        delay = []
        for a in atomix_list:
            delay.append(distance_matrix[c][a])
        average_atomix_delay_from[c] = np.mean(delay)

    # find the nearest atomix for each atomix and calculate average delay
    atomix_atomix_delays = []
    for a1 in atomix_list:
        delay = math.inf
        for a2 in atomix_list:
            if(a1 == a2):
                continue
            if distance_matrix[a1][a2] < delay:
                delay = distance_matrix[a1][a2]
        atomix_atomix_delays.append(delay)
    average_atomix_atomix_delay = np.mean(atomix_atomix_delays)
    FTSs = []
    for source in range(num_nodes):
        for distination in range(num_nodes):
            if(source == distination):
                continue
            delay = 0
            is_controlled_by_single_controller = True
            counted_controllers = []
            for s in shortest_paths[source][distination]:
                # switch-controller delay
                delay += distance_matrix[s][controller_of[s]] * 4

                # controller-atomix delay
                if(s == source):
                    delay += average_atomix_delay_from[controller_of[s]] * 2
                elif(s != distination):
                    if(controller_of[s] != controller_of[source]):
                        is_controlled_by_single_controller = False
                        if(not controller_of[s] in counted_controllers):
                            counted_controllers.append(controller_of[s])
                            delay += average_atomix_delay_from[controller_of[s]]
                else:
                    if(controller_of[s] == controller_of[source]):
                        if(not is_controlled_by_single_controller):
                            delay += average_atomix_delay_from[controller_of[s]]
                    else:
                        delay += average_atomix_delay_from[controller_of[s]] * 2
            
            # atomix-atomix delay
            delay +=  average_atomix_atomix_delay * 2
            FTSs.append(delay)

    return np.mean(FTSs)



def calculate_BC(num_nodes, controller_nodes, atomix_nodes, distance_matrix, graph):
    G = nx.Graph()
    for node1 in range(len(graph)):
        G.add_node(str(node1))
        for node2, delay in graph[node1].items():
            G.add_edge(str(node1), str(node2), weight=delay)
    
    # The list of betweenness centrality for all switches
    nodes_bc=nx.current_flow_betweenness_centrality(G, normalized=True, weight=None, dtype='float', solver='full')
    num_controller = np.sum(controller_nodes)
    num_atomix = np.sum(atomix_nodes)
    controller_list = np.nonzero(controller_nodes)[0].tolist()

    if(num_controller == 0 or num_atomix ==0):
        return math.inf

    # find the nearest controller for each switch
    controller_of = []
    limit_num_switches_controlled=int(math.ceil(num_nodes/num_controller)) # balance the number of switches controllers can control 
    switches_bc_of_controller_ = dict.fromkeys((range(num_nodes)),0) # list of sum of betweenness centrality of switches for each controller
    for s in range(num_nodes):
        delay = math.inf
        nearest_controller = None
        controlled_switches=[]
        for c in controller_list:
            # Conditions: nearest controller (with the lowest delay) && the number of switches for each controller < limit_num_switches_controlled
            if distance_matrix[s][c] < delay and controller_of.count(c) < limit_num_switches_controlled:
                delay = distance_matrix[s][c]
                nearest_controller = c
                controlled_switches.append(s)
        switches_bc_of_controller_[nearest_controller] += nodes_bc[str(s)]
        controller_of.append(nearest_controller)
    
    # Simplify switches_bc_of_controller_ (only need value for calculating variance)
    bc_array = []
    for i in switches_bc_of_controller_.values():
        bc_array.append(i)

    # return variance value can show the degree of balance within all controllers
    return np.var(bc_array)

### Load moead results for all topos

In [2]:
with open('res_bc_Cogent_moead.pkl','rb') as f_Cogent:
    res_13_Cogent = pickle.load(f_Cogent)
with open('res_bc_UsCarrier_moead.pkl','rb') as f_UsCarrier:
    res_13_UsCarrier = pickle.load(f_UsCarrier)
with open('res_bc_HiberniaGlobal_moead.pkl','rb') as f_HiberniaGlobal:
    res_13_HiberniaGlobal = pickle.load(f_HiberniaGlobal)
with open('res_bc_Colt_moead.pkl','rb') as f_Colt:
    res_13_Colt = pickle.load(f_Colt)
with open('res_bc_Funet_moead.pkl','rb') as f_Funet:
    res_13_Funet = pickle.load(f_Funet)
with open('res_bc_Abvt_moead.pkl','rb') as f_Abvt:
    res_13_Abvt = pickle.load(f_Abvt)
with open('res_bc_Intellifiber_moead.pkl','rb') as f_Intellifiber:
    res_13_Intellifiber = pickle.load(f_Intellifiber)
with open('res_bc_TataNld_moead.pkl','rb') as f_TataNld:
    res_13_TataNld = pickle.load(f_TataNld)
with open('res_bc_Internode_moead.pkl','rb') as f_Internode:
    res_13_Internode = pickle.load(f_Internode)
with open('res_bc_Missouri_moead.pkl','rb') as f_Missouri:
    res_13_Missouri = pickle.load(f_Missouri)
with open('res_bc_Ion_moead.pkl','rb') as f_Ion:
    res_13_Ion = pickle.load(f_Ion)
with open('res_bc_Palmetto_moead.pkl','rb') as f_Palmetto:
    res_13_Palmetto = pickle.load(f_Palmetto)

## Hypervolume
1. Store values

In [ ]:
# moead
F13_Cogent=res_13_Cogent.F
F13_UsCarrier=res_13_UsCarrier.F
F13_HiberniaGlobal=res_13_HiberniaGlobal.F
F13_Colt=res_13_Colt.F
F13_Funet=res_13_Funet.F
F13_Abvt=res_13_Abvt.F
F13_Intellifiber=res_13_Intellifiber.F
F13_TataNld=res_13_TataNld.F
F13_Internode=res_13_Internode.F
F13_Missouri=res_13_Missouri.F
F13_Ion=res_13_Ion.F
F13_Palmetto=res_13_Palmetto.F

# # Nadir Point from 9 Algorithms: 
# ref_point = [1.81000000e+02 6.20000000e+01 3.92281875e+03 1.70537209e+00]
# # Nadir Point from 4 Algorithms: 
# ref_point = [160.  77.  inf  inf]
# # Nadir Point from all (13) Algorithms: 
# ref_point = [181.  77.  inf  inf]

ref_point = [181, 77, 3922.81875, 1.70537209]

hist_F13_Cogent = []
hist_F13_UsCarrier = []
hist_F13_HiberniaGlobal = []
hist_F13_Colt = []
hist_F13_Funet = []
hist_F13_Abvt = []
hist_F13_Intellifiber = []
hist_F13_TataNld = []
hist_F13_Internode = []
hist_F13_Missouri = []
hist_F13_Ion = []
hist_F13_Palmetto = []


for algo13_Cogent in res_13_Cogent.history:
    opt13_Cogent = algo13_Cogent.opt
    feas13_Cogent = np.where(opt13_Cogent.get("feasible"))[0]
    hist_F13_Cogent.append(opt13_Cogent.get("F")[feas13_Cogent])
for algo13_UsCarrier in res_13_UsCarrier.history:
    opt13_UsCarrier = algo13_UsCarrier.opt
    feas13_UsCarrier = np.where(opt13_UsCarrier.get("feasible"))[0]
    hist_F13_UsCarrier.append(opt13_UsCarrier.get("F")[feas13_UsCarrier])
for algo13_HiberniaGlobal in res_13_HiberniaGlobal.history:
    opt13_HiberniaGlobal = algo13_HiberniaGlobal.opt
    feas13_HiberniaGlobal = np.where(opt13_HiberniaGlobal.get("feasible"))[0]
    hist_F13_HiberniaGlobal.append(opt13_HiberniaGlobal.get("F")[feas13_HiberniaGlobal])
for algo13_Colt in res_13_Colt.history:
    opt13_Colt = algo13_Colt.opt
    feas13_Colt = np.where(opt13_Colt.get("feasible"))[0]
    hist_F13_Colt.append(opt13_Colt.get("F")[feas13_Colt])
for algo13_Funet in res_13_Funet.history:
    opt13_Funet = algo13_Funet.opt
    feas13_Funet = np.where(opt13_Funet.get("feasible"))[0]
    hist_F13_Funet.append(opt13_Funet.get("F")[feas13_Funet])
for algo13_Abvt in res_13_Abvt.history:
    opt13_Abvt = algo13_Abvt.opt
    feas13_Abvt = np.where(opt13_Abvt.get("feasible"))[0]
    hist_F13_Abvt.append(opt13_Abvt.get("F")[feas13_Abvt])
for algo13_Intellifiber in res_13_Intellifiber.history:
    opt13_Intellifiber = algo13_Intellifiber.opt
    feas13_Intellifiber = np.where(opt13_Intellifiber.get("feasible"))[0]
    hist_F13_Intellifiber.append(opt13_Intellifiber.get("F")[feas13_Intellifiber])
for algo13_TataNld in res_13_TataNld.history:
    opt13_TataNld = algo13_TataNld.opt
    feas13_TataNld = np.where(opt13_TataNld.get("feasible"))[0]
    hist_F13_TataNld.append(opt13_TataNld.get("F")[feas13_TataNld])
for algo13_Internode in res_13_Internode.history:
    opt13_Internode = algo13_Internode.opt
    feas13_Internode= np.where(opt13_Internode.get("feasible"))[0]
    hist_F13_Internode.append(opt13_Internode.get("F")[feas13_Internode])
for algo13_Missouri in res_13_Missouri.history:
    opt13_Missouri = algo13_Missouri.opt
    feas13_Missouri = np.where(opt13_Missouri.get("feasible"))[0]
    hist_F13_Missouri.append(opt13_Missouri.get("F")[feas13_Missouri])
for algo13_Ion in res_13_Ion.history:
    opt13_Ion = algo13_Ion.opt
    feas13_Ion = np.where(opt13_Ion.get("feasible"))[0]
    hist_F13_Ion.append(opt13_Ion.get("F")[feas13_Ion])
for algo13_Palmetto in res_13_Palmetto.history:
    opt13_Palmetto = algo13_Palmetto.opt
    feas13_Palmetto = np.where(opt13_Palmetto.get("feasible"))[0]
    hist_F13_Palmetto.append(opt13_Palmetto.get("F")[feas13_Palmetto])

2. Calculate hypervolume and merge them (Mean value) for moead algorithm

In [ ]:
from pymoo.indicators.hv import Hypervolume

metric = Hypervolume(ref_point= np.array(ref_point))

hv_13_Cogent = [metric.do(_F13_Cogent) for _F13_Cogent in hist_F13_Cogent]
hv_13_UsCarrier = [metric.do(_F13_UsCarrier) for _F13_UsCarrier in hist_F13_UsCarrier]
hv_13_HiberniaGlobal = [metric.do(_F13_HiberniaGlobal) for _F13_HiberniaGlobal in hist_F13_HiberniaGlobal]
hv_13_Colt = [metric.do(_F13_Colt) for _F13_Colt in hist_F13_Colt]
hv_13_Funet = [metric.do(_F13_Funet) for _F13_Funet in hist_F13_Funet]
hv_13_Abvt = [metric.do(_F13_Abvt) for _F13_Abvt in hist_F13_Abvt]
hv_13_Intellifiber = [metric.do(_F13_Intellifiber) for _F13_Intellifiber in hist_F13_Intellifiber]
hv_13_TataNld = [metric.do(_F13_TataNld) for _F13_TataNld in hist_F13_TataNld]
hv_13_Internode = [metric.do(_F13_Internode) for _F13_Internode in hist_F13_Internode]
hv_13_Missouri = [metric.do(_F13_Missouri) for _F13_Missouri in hist_F13_Missouri]
hv_13_Ion = [metric.do(_F13_Ion) for _F13_Ion in hist_F13_Ion]
hv_13_Palmetto = [metric.do(_F13_Palmetto) for _F13_Palmetto in hist_F13_Palmetto]

# moead
hv_13 = [(hv_Cogent + hv_UsCarrier + hv_HiberniaGlobal + hv_Colt + hv_Funet + hv_Abvt + hv_Intellifiber + hv_TataNld + hv_Internode + hv_Missouri + hv_Ion + hv_Palmetto) / 12 for hv_Cogent, hv_UsCarrier, hv_HiberniaGlobal, hv_Colt, hv_Funet, hv_Abvt, hv_Intellifiber, hv_TataNld, hv_Internode, hv_Missouri, hv_Ion, hv_Palmetto in zip(hv_13_Cogent, hv_13_UsCarrier, hv_13_HiberniaGlobal, hv_13_Colt, hv_13_Funet, hv_13_Abvt, hv_13_Intellifiber, hv_13_TataNld, hv_13_Internode, hv_13_Missouri, hv_13_Ion, hv_13_Palmetto)]

In [ ]:
print('The hypervolumn for Cogent:')
print(hv_13_Cogent)
print('The hypervolumn for UsCarrier:')
print(hv_13_UsCarrier)
print('The hypervolumn for HiberniaGlobal:')
print(hv_13_HiberniaGlobal)
print('The hypervolumn for Colt:')
print(hv_13_Colt)
print('The hypervolumn for Funet:')
print(hv_13_Funet)
print('The hypervolumn for Abvt:')
print(hv_13_Abvt)
print('The hypervolumn for Intellifiber:')
print(hv_13_Intellifiber)
print('The hypervolumn for TataNld:')
print(hv_13_TataNld)
print('The hypervolumn for Internode:')
print(hv_13_Internode)
print('The hypervolumn for Missouri:')
print(hv_13_Missouri)
print('The hypervolumn for Ion:')
print(hv_13_Ion)
print('The hypervolumn for Palmetto:')
print(hv_13_Palmetto)

# Show all hv for each topo
import matplotlib.pyplot as plt

plt.figure(figsize=(7, 5))
plt.plot(list(range(1, len(hv_13_Cogent)+1)), hv_13_Cogent,  color='black', label='Cogent')
plt.plot(list(range(1, len(hv_13_UsCarrier)+1)), hv_13_UsCarrier,  color='blue', label='UsCarrier')
plt.plot(list(range(1, len(hv_13_HiberniaGlobal)+1)), hv_13_HiberniaGlobal,  color='red', label='HiberniaGlobal')
plt.plot(list(range(1, len(hv_13_Colt)+1)), hv_13_Colt,  color='orange', label='Colt')
plt.plot(list(range(1, len(hv_13_Funet)+1)), hv_13_Funet,  color=(0, 0, 1, 0.5), label='Funet')
plt.plot(list(range(1, len(hv_13_Abvt)+1)), hv_13_Abvt,  color='gray', label='Abvt')
plt.plot(list(range(1, len(hv_13_Intellifiber)+1)), hv_13_Intellifiber,  color='purple', label='Intellifiber')
plt.plot(list(range(1, len(hv_13_TataNld)+1)), hv_13_TataNld,  color='green', label='TataNld')
plt.plot(list(range(1, len(hv_13_Internode)+1)), hv_13_Internode,  color='#33FF57', label='Internode')
plt.plot(list(range(1, len(hv_13_Missouri)+1)), hv_13_Missouri,  color='brown', label='Missouri')
plt.plot(list(range(1, len(hv_13_Ion)+1)), hv_13_Ion,  color='pink', label='Ion')
plt.plot(list(range(1, len(hv_13_Palmetto)+1)), hv_13_Palmetto,  color='olive', label='Palmetto')
plt.title("Convergence")
plt.xlabel("Topos")
plt.ylabel("Hypervolume")
plt.legend()
plt.show()

In [ ]:
print('The aveage of hypervolumn for moead:')
print(hv_13)

3. Draw mean hypervolume for moead

In [ ]:
import matplotlib.pyplot as plt

#  Show all alogrithms' hv into a single figure
plt.figure(figsize=(7, 5))
# plt.plot(list(range(1, len(hv_1)+1)), hv_1,  color='black', label='AGEMOEA')
# plt.plot(list(range(1, len(hv_2)+1)), hv_2,  color='blue', label='AGEMOEA2')
# plt.plot(list(range(1, len(hv_3)+1)), hv_3,  color='red', label='NSGA2')
# plt.plot(list(range(1, len(hv_4)+1)), hv_4,  color='orange', label='NSGA3')
# plt.plot(list(range(1, len(hv_5)+1)), hv_5,  color=(0, 0, 1, 0.5), label='RNSGA2')
# plt.plot(list(range(1, len(hv_6)+1)), hv_6,  color='gray', label='RNSGA3')
# plt.plot(list(range(1, len(hv_7)+1)), hv_7,  color='purple', label='RVEA')
# plt.plot(list(range(1, len(hv_8)+1)), hv_8,  color='green', label='SMSEMOA')
# plt.plot(list(range(1, len(hv_9)+1)), hv_9,  color='#33FF57', label='UNSGA3')
# plt.plot(list(range(1, len(hv_10)+1)), hv_10,  color='brown', label='CTAEA')
# plt.plot(list(range(1, len(hv_11)+1)), hv_11,  color='pink', label='DNSGA2')
# plt.plot(list(range(1, len(hv_12)+1)), hv_12,  color='olive', label='KGBDMOEA')
plt.plot(list(range(1, len(hv_13)+1)), hv_13,  color='cyan', label='MOEAD')
plt.title("Convergence")
plt.xlabel("Generations") # brown,pink,#5733FF,olive,cyan,#FF5733,#33FF57
plt.ylabel("Hypervolume")
plt.legend()
plt.show()